In [ ]:
# importar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Procesamiento de Datos

In [ ]:
# obtener el conjunto de datos
df = pd.read_csv('data.csv')

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
df["Diet"].unique()

In [ ]:
# Verify if there are any null values
print(df.isna().sum())

## Transformación de Datos


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Seleccionar las características relevantes para el SOM (Age, Cholesterol, Blood Pressure, ...)
features = [
    # "Patient ID",
    "Age",
    "Sex",
    "Blood Pressure",
    "Smoking",
    # "Cholesterol",
    "Heart Rate",
    # "Diabetes",
    # "Family History",
    "Alcohol Consumption",
    "Exercise Hours Per Week",
    "Diet",
    # "Previous Heart Problems",
    # "Sedentary Hours Per Day",
    "BMI",
    # "Physical Activity Days Per Week",
    # "Sleep Hours Per Day",
    "Heart Attack Risk",
]
df = df[features]

# Dividir la columna 'Blood Pressure' en dos columnas separadas
df[["Systolic Pressure", "Diastolic Pressure"]] = df["Blood Pressure"].str.split(
    "/", expand=True
)
df["Systolic Pressure"] = pd.to_numeric(df["Systolic Pressure"])
df["Diastolic Pressure"] = pd.to_numeric(df["Diastolic Pressure"])
df.drop(columns=["Blood Pressure"], inplace=True)

# Convertir la columna "Diet" (['Average', 'Unhealthy', 'Healthy']) a numerico
df["Diet"] = df["Diet"].apply(
    lambda x: 0 if x == "Unhealthy" else 1 if x == "Average" else 2
)

# Convertir las columnas a valores numéricos
# df["Patient ID"] = df["Patient ID"].apply(lambda x: x[3:])

# Convertir las columnas a valores numéricos
df["Sex"] = df["Sex"].apply(lambda x: 1 if x == "Male" else 0)

In [ ]:
# graph of the corr
import seaborn as sns
plt.figure(figsize=(10, 10))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.show()

# Modelate

In [ ]:
df.head()

## Split Data

In [ ]:
# Split the dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    ConfusionMatrixDisplay,
)


X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Heart Attack Risk", axis=1),
    df["Heart Attack Risk"],
    random_state=0,
    test_size=0.2,
)

In [ ]:
# Eliminamos el Id del pasajero en el conjunto de datos de prueba
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Model Perceptron


In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
# Perceptron
perceptron = Perceptron()
perceptron.fit(X_train, y_train)
predictions = perceptron.predict(X_test)
# Calculate his accuracy of the perceptron
acc_perceptron = accuracy_score(y_test, predictions)
# Print the accuracy
print(acc_perceptron)

## Model Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
# print accuracy
acc_gnb =accuracy_score(y_test, y_pred)
print(acc_gnb)

## Model Regresion Logistic

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier


### SGDClassifier

In [ ]:
logistic_model_SGD = SGDClassifier(loss='log_loss',learning_rate='constant',eta0=0.1 ) # investicar los parámetros en la documentacion y variar el learning_rate
logistic_model_SGD.fit(X_train, y_train)

In [ ]:
print(f'Clases de la variable dependiente: {logistic_model_SGD.classes_}')
print('\n')
print('Vectores de coeficientes:')
print(logistic_model_SGD.coef_)

In [ ]:
y_pred = logistic_model_SGD.predict(X_train)

In [ ]:
print(f'Accuracy entrenamiento: {accuracy_score(y_train, y_pred)}')
print('Matriz de confusión:')
matriz_confusion = confusion_matrix(y_train, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=matriz_confusion, display_labels=logistic_model_SGD.classes_)
disp.plot()
plt.show()

In [ ]:
y_pred_test = logistic_model_SGD.predict(X_test)

In [ ]:
print(f'Accuracy testing: {accuracy_score(y_test, y_pred_test)}')
print('Matriz de confusión:')
matriz_confusion_test = confusion_matrix(y_test, y_pred_test)
disp = ConfusionMatrixDisplay(confusion_matrix=matriz_confusion_test, display_labels=logistic_model_SGD.classes_)
disp.plot()
plt.show()

### LogisticRegression

In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

In [ ]:
print(f'Clases de la variable dependiente: {logistic_model.classes_}')
print('\n')
print('Vectores de coeficientes:')
print(logistic_model.coef_)

In [ ]:
y_pred_2 = logistic_model.predict(X_train)

In [ ]:
print(f'Accuracy entrenamiento: {accuracy_score(y_train, y_pred_2)}')
print('Matriz de confusión:')
matriz_confusion_2 = confusion_matrix(y_train, y_pred_2)
disp = ConfusionMatrixDisplay(confusion_matrix=matriz_confusion_2, display_labels=logistic_model.classes_)
disp.plot()
plt.show()

In [ ]:
y_pred_test_2 = logistic_model.predict(X_test)

In [ ]:
print(f'Accuracy testing: {accuracy_score(y_test, y_pred_test_2)}')
print('Matriz de confusión:')
matriz_confusion_test_2 = confusion_matrix(y_test, y_pred_test_2)
disp = ConfusionMatrixDisplay(confusion_matrix=matriz_confusion_test_2, display_labels=logistic_model.classes_)
disp.plot()
plt.show()

In [ ]:
# Use network neural
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
X_train.shape

In [ ]:
## Con sequetial podemos construir una red neuronal apilando capas
modelsequ = Sequential()
modelsequ.add(tf.keras.Input(shape=(10,), dtype=tf.string))
modelsequ.add(Dense(10, activation='relu'))
modelsequ.add(Dense(1, activation='sigmoid'))
modelsequ.compile(loss='binary_crossentropy',  optimizer='adam', metrics=['accuracy'])
modelsequ.summary()

In [ ]:
historyseq = modelsequ.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=500
)